In [8]:
import os
import time
from datetime import datetime, timezone
import pandas as pd
import schedule  # pip install schedule

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

symbollist = pd.read_csv("/Users/michal/PycharmProjects/Stock Scraper/sp500symbols.csv")

base_url = "https://edition.cnn.com/markets/stocks/"

# Tworzenie listy "stocks" z adresami URL
stocks = [base_url + symbol for symbol in symbollist["Symbol"]]

# Sprawdzenie unikalności
if len(stocks) == len(set(stocks)):
    print("Wszystkie elementy w liście są unikalne.")
else:
    print("W liście znajdują się duplikaty.")
    # Wyszukanie duplikatów
    duplicates = [item for item in stocks if stocks.count(item) > 1]
    print("Duplikaty:", set(duplicates))

    # Usuwanie duplikatów
    stocks = list(set(stocks))
    print("Usunięto duplikaty:", set(duplicates))

print("Liczba monitorowanych spółek - po obróbce duplikatów:", len(stocks))

print()
print("-" * 57)

# Ścieżka do folderu, gdzie zapisujemy plik CSV
OUTPUT_CSV_PATH = "/Users/michal/PycharmProjects/Stock Scraper/stocks/stocks_data.csv"


# Ścieżka do WebDrivera
driver_path = "/Users/michal/Downloads/chromedriver-mac-x64/chromedriver"


# Pobieranie aktualnego czasu UTC
current_time = datetime.now(timezone.utc)

# Wyodrębnienie daty i godziny
hour_timestamp = current_time.strftime("%H:%M:%S UTC")
date_timestamp = current_time.strftime("%Y-%m-%d")


def scrape_data(limit):
    """
    Funkcja, która iteruje po wszystkich spółkach w 'stocks',
    pobiera dane i zwraca je jako DataFrame.
    """
    all_data = []  # tutaj będziemy przechowywać słowniki z danymi każdej spółki
    drops = []

    # Konfiguracja Selenium (otwieramy raz przeglądarkę na czas całego procesu)
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    wait = WebDriverWait(driver, 30)

    # Iteracja po spółkach z listy "stocks"
    for index, url in enumerate(stocks[:limit], start=1):
        symbol = url.split("/")[-1]
        print(f"Procesuję spółkę {index}/{limit}: {symbol}")

        # Przygotowanie słownika na dane
        stats = {}

        try:
            # Mierzenie czasu ładowania strony
            start_time = time.time()
            driver.get(url)
            end_time = time.time()

            # Obliczenie czasu ładowania strony
            load_time = end_time - start_time
            print(f"Czas ładowania strony {symbol}: {load_time:.2f} sekund")

            # Dodatkowy czas oczekiwania po załadowaniu strony
            time.sleep(1)

            # Czekamy na kluczowy element strony zamiast time.sleep()
            try:
                dynamic_element_wait_start = time.time()
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CLASS_NAME, "price-2kQQGw.cnn-pcl-eltrz4")
                    )
                )
                dynamic_element_wait_end = time.time()
                dynamic_load_time = dynamic_element_wait_end - dynamic_element_wait_start
                print(f"Czas oczekiwania na dynamiczne elementy {symbol}: {dynamic_load_time:.2f} sekund")
                print()
                print("-" * 57)

            except Exception as e:
                print(f"Dynamiczne elementy strony {symbol} nie załadowały się w wyznaczonym czasie.")
                dynamic_load_time = None
                # Łączny czas oczekiwania
                total_wait_time = load_time + (dynamic_load_time if dynamic_load_time else 0)
                print(f"Łączny czas oczekiwania {symbol}: {total_wait_time:.2f} sekund")
                print()
                print("-" * 57)

            # 0. Czas ładowania strony
            stats["Page Load Time (s)"] = round(load_time, 2)
            # Jeśli dynamic_load_time się nie ustawiło (np. w except), chcemy uniknąć błędu:
            if 'dynamic_load_time' not in locals() or dynamic_load_time is None:
                dynamic_load_time = float('nan')  # lub 0.0, w zależności od preferencji
            stats["Dynamic Element Load Time (s)"] = round(dynamic_load_time, 2)

            # 1. Cena akcji
            price_element = driver.find_element(By.CLASS_NAME, "price-2kQQGw.cnn-pcl-eltrz4")
            price = price_element.text
            stats["Price"] = price

            # 2. Zmiana ceny
            change_element = driver.find_element(By.CLASS_NAME, "sub-price-1huDfE.cnn-pcl-eltrz4")
            change = change_element.text
            stats["Change"] = change

            # 3. Zmiana procentowa
            percent_change_element = driver.find_element(
                By.CLASS_NAME,
                "sub-price-1huDfE.percent-21eK0W.cnn-pcl-eltrz4"
            )
            percent_change = percent_change_element.text
            stats["Percent Change"] = percent_change

            # 4. Data sesji
            session_date_element = driver.find_element(
                By.CLASS_NAME,
                "timestamp-2-ZRU_.cnn-pcl-eltrz4"
            )
            session_date = session_date_element.text
            stats["Session Date"] = session_date

            # 5. Cena po zamknięciu - to działa tylko jak sesja jest już zamknięta
            closing_price_element = driver.find_element(
                By.CSS_SELECTOR,
                ".pricing-container-3_SSKi.secondary-160A5Y "
                ".price-2kQQGw.secondary-160A5Y.cnn-pcl-eltrz4"
            )
            closing_price = closing_price_element.text
            stats["Closing Price"] = closing_price

            # 6. Key stock statistics
            keys = driver.find_elements(By.CLASS_NAME, "markets-keyfacts__label-33Y3Fm")
            values = driver.find_elements(By.CLASS_NAME, "markets-keyfacts__value-3a2Zj8")
            for key, value in zip(keys, values):
                key_text = key.text.strip()
                value_text = value.text.strip()
                stats[key_text] = value_text

            # 7. Smart Score z iframe TipRanks
            try:
                iframe = driver.find_element(By.CSS_SELECTOR, "iframe[src*='tipranks']")
                driver.switch_to.frame(iframe)
                smart_score_element = driver.find_element(By.CLASS_NAME, "sc-cGXZpB.hINHXp")
                smart_score = smart_score_element.text
                stats["Smart Score"] = smart_score
            except Exception as iframe_error:
                # Jeżeli nie znajdziemy iframe, wstawiamy None
                stats["Smart Score"] = None
            finally:
                # powrót do głównej strony
                driver.switch_to.default_content()

            # 8. Dane finansowe (Revenue, Net Income, itp.)
            financial_rows = driver.find_elements(
                By.CLASS_NAME,
                "market-financial-table__row-153LbB"
            )
            for row in financial_rows:
                row_title = row.find_element(
                    By.CLASS_NAME,
                    "market-financial-table__title"
                ).text.strip()
                row_value = row.find_element(
                    By.CLASS_NAME,
                    "market-financial-table__text"
                ).text.strip()
                row_change = row.find_element(
                    By.CLASS_NAME,
                    "market-financial-table__change"
                ).text.strip()

                stats[f"{row_title} Value"] = row_value
                stats[f"{row_title} Change"] = row_change

            # 9. Dodaj identyfikator spółki
            stats["Stock"] = symbol

            # 10. Analyst Ratings
            time.sleep(2)
            number_of_analysts = wait.until(
                EC.presence_of_element_located(
                    (By.CLASS_NAME, "markets-donut-chart__title")
                )
            ).text
            stats["Number of analysts"] = number_of_analysts

            buy_recommendation = wait.until(
                EC.presence_of_element_located(
                    (By.ID, "markets-donut-chart__legend--key-value-buy")
                )
            ).text
            hold_recommendation = wait.until(
                EC.presence_of_element_located(
                    (By.ID, "markets-donut-chart__legend--key-value-hold")
                )
            ).text
            sell_recommendation = wait.until(
                EC.presence_of_element_located(
                    (By.ID, "markets-donut-chart__legend--key-value-sell")
                )
            ).text

            stats["Buy Recommendation"] = buy_recommendation
            stats["Hold Recommendation"] = hold_recommendation
            stats["Sell Recommendation"] = sell_recommendation

            # 11. Forecast (High/Median/Low)
            driver.execute_script("window.scrollTo(0, 2500);")
            time.sleep(2)
            forecast_chart_section = wait.until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "div.markets-forecast-chart")
                )
            )
            svg_element = forecast_chart_section.find_element(By.CSS_SELECTOR, "svg")

            # High Forecast
            try:
                high_data_g = svg_element.find_element(By.CSS_SELECTOR, "g.high-data")
                high_price_tspans = high_data_g.find_elements(
                    By.CSS_SELECTOR,
                    "text.high-price tspan"
                )
                if len(high_price_tspans) >= 2:
                    stats["High Forecast"] = high_price_tspans[1].text.strip()
                else:
                    stats["High Forecast"] = None
            except:
                stats["High Forecast"] = None

            # Median Forecast
            try:
                median_data_g = svg_element.find_element(By.CSS_SELECTOR, "g.median-data")
                median_price_tspans = median_data_g.find_elements(
                    By.CSS_SELECTOR,
                    "text.median-price tspan"
                )
                if len(median_price_tspans) >= 2:
                    stats["Median Forecast"] = median_price_tspans[1].text.strip()
                else:
                    stats["Median Forecast"] = None
            except:
                stats["Median Forecast"] = None

            # Low Forecast
            try:
                low_data_g = svg_element.find_element(By.CSS_SELECTOR, "g.low-data")
                low_price_tspans = low_data_g.find_elements(
                    By.CSS_SELECTOR,
                    "text.median-price tspan"
                )
                if len(low_price_tspans) >= 2:
                    stats["Low Forecast"] = low_price_tspans[1].text.strip()
                else:
                    stats["Low Forecast"] = None
            except:
                stats["Low Forecast"] = None

            # Dodaj do listy all_data
            all_data.append(stats)

        except Exception as e:
            print(f"Wystąpił błąd podczas przetwarzania spółki {symbol}.")
            drops.append(symbol)
            print(f"Dopisano spółkę {symbol} do listy ponownego przetworzenia.")
            print(e)
            print("-" * 57)

    # Zamykamy driver po zakończeniu pętli
    driver.quit()

    # Tworzymy DataFrame z listy słowników
    df = pd.DataFrame(all_data)

    # # Pobieranie aktualnego czasu UTC
    # current_time = datetime.now(timezone.utc)
    # 
    # # Wyodrębnienie daty i godziny
    # hour_timestamp = current_time.strftime("%H:%M:%S UTC")
    # date_timestamp = current_time.strftime("%Y-%m-%d")

    # Dodanie kolumn do DataFrame
    df["Time of record"] = hour_timestamp
    df["Date of record"] = date_timestamp

    return df, drops


def retry_drops(drops):
    """
    Funkcja, która iteruje po spółkach z listy 'drops',
    próbuje ponownie pobrać dane i zwraca DataFrame oraz nową listę 'drops'.
    
    Parametry:
    - drops (list): Lista symboli spółek do ponownego przetworzenia.
    
    Zwraca:
    - tuple: (DataFrame z danymi przetworzonych spółek, nowa lista 'drops')
    """
    all_data = []      # Przechowuje dane każdej spółki
    new_drops = []     # Przechowuje spółki, które nadal napotykają błędy
    
    # Konfiguracja Selenium (otwieramy przeglądarkę na czas procesu)
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    wait = WebDriverWait(driver, 30)
    
    for index, symbol in enumerate(drops, start=1):
        # Znajdź URL odpowiadający symbolowi
        try:
            url = next(url for url in stocks if url.split("/")[-1] == symbol)
        except StopIteration:
            print(f"URL dla symbolu {symbol} nie został znaleziony w liście 'stocks'.")
            new_drops.append(symbol)
            continue
        
        print(f"Ponowna próba pobierania spółki {index}/{len(drops)}: {symbol}")
        
        # Przygotowanie słownika na dane
        stats = {}
    
        try:
            # Mierzenie czasu ładowania strony
            start_time = time.time()
            driver.get(url)
            end_time = time.time()
    
            # Obliczenie czasu ładowania strony
            load_time = end_time - start_time
            print(f"Czas ładowania strony {symbol}: {load_time:.2f} sekund")
    
            # Dodatkowy czas oczekiwania po załadowaniu strony
            time.sleep(1)
    
            # Czekamy na kluczowy element strony zamiast time.sleep()
            try:
                dynamic_element_wait_start = time.time()
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CLASS_NAME, "price-2kQQGw.cnn-pcl-eltrz4")
                    )
                )
                dynamic_element_wait_end = time.time()
                dynamic_load_time = dynamic_element_wait_end - dynamic_element_wait_start
                print(f"Czas oczekiwania na dynamiczne elementy {symbol}: {dynamic_load_time:.2f} sekund")
                print("-" * 57)
            except Exception as e:
                print(f"Dynamiczne elementy strony {symbol} nie załadowały się w wyznaczonym czasie.")
                dynamic_load_time = float('nan')  # Ustaw NaN jeśli nie załadowały się
                print(f"Łączny czas oczekiwania {symbol}: {load_time + (dynamic_load_time if not pd.isna(dynamic_load_time) else 0):.2f} sekund")
                print("-" * 57)
    
            # 0. Czas ładowania strony
            stats["Page Load Time (s)"] = round(load_time, 2)
            # 1. Czas oczekiwania na dynamiczne elementy
            stats["Dynamic Element Load Time (s)"] = round(dynamic_load_time, 2)
    
            # 1. Cena akcji
            price_element = driver.find_element(By.CLASS_NAME, "price-2kQQGw.cnn-pcl-eltrz4")
            price = price_element.text
            stats["Price"] = price
    
            # 2. Zmiana ceny
            change_element = driver.find_element(By.CLASS_NAME, "sub-price-1huDfE.cnn-pcl-eltrz4")
            change = change_element.text
            stats["Change"] = change
    
            # 3. Zmiana procentowa
            percent_change_element = driver.find_element(
                By.CLASS_NAME,
                "sub-price-1huDfE.percent-21eK0W.cnn-pcl-eltrz4"
            )
            percent_change = percent_change_element.text
            stats["Percent Change"] = percent_change
    
            # 4. Data sesji
            session_date_element = driver.find_element(
                By.CLASS_NAME,
                "timestamp-2-ZRU_.cnn-pcl-eltrz4"
            )
            session_date = session_date_element.text
            stats["Session Date"] = session_date
    
            # 5. Cena po zamknięciu - to działa tylko jak sesja jest już zamknięta
            closing_price_element = driver.find_element(
                By.CSS_SELECTOR,
                ".pricing-container-3_SSKi.secondary-160A5Y "
                ".price-2kQQGw.secondary-160A5Y.cnn-pcl-eltrz4"
            )
            closing_price = closing_price_element.text
            stats["Closing Price"] = closing_price
    
            # 6. Key stock statistics
            keys = driver.find_elements(By.CLASS_NAME, "markets-keyfacts__label-33Y3Fm")
            values = driver.find_elements(By.CLASS_NAME, "markets-keyfacts__value-3a2Zj8")
            for key, value in zip(keys, values):
                key_text = key.text.strip()
                value_text = value.text.strip()
                stats[key_text] = value_text
    
            # 7. Smart Score z iframe TipRanks
            try:
                iframe = driver.find_element(By.CSS_SELECTOR, "iframe[src*='tipranks']")
                driver.switch_to.frame(iframe)
                smart_score_element = driver.find_element(By.CLASS_NAME, "sc-cGXZpB.hINHXp")
                smart_score = smart_score_element.text
                stats["Smart Score"] = smart_score
            except Exception as iframe_error:
                # Jeżeli nie znajdziemy iframe, wstawiamy NaN
                stats["Smart Score"] = float('nan')
                print(f"Brak danych Smart Score dla spółki {symbol}.")
                print(iframe_error)
            finally:
                # Powrót do głównej strony
                driver.switch_to.default_content()
    
            # 8. Dane finansowe (Revenue, Net Income, itp.)
            try:
                financial_rows = driver.find_elements(
                    By.CLASS_NAME,
                    "market-financial-table__row-153LbB"
                )
                for row in financial_rows:
                    try:
                        row_title = row.find_element(
                            By.CLASS_NAME,
                            "market-financial-table__title"
                        ).text.strip()
                        row_value = row.find_element(
                            By.CLASS_NAME,
                            "market-financial-table__text"
                        ).text.strip()
                        row_change = row.find_element(
                            By.CLASS_NAME,
                            "market-financial-table__change"
                        ).text.strip()
    
                        stats[f"{row_title} Value"] = row_value
                        stats[f"{row_title} Change"] = row_change
                    except Exception as e:
                        print(f"Brak pełnych danych finansowych w wierszu dla spółki {symbol}.")
                        print(e)
            except Exception as e:
                print(f"Brak danych finansowych dla spółki {symbol}.")
                print(e)
    
            # 9. Dodaj identyfikator spółki
            stats["Stock"] = symbol
    
            # 10. Analyst Ratings
            try:
                time.sleep(1)
                number_of_analysts = wait.until(
                    EC.presence_of_element_located(
                        (By.CLASS_NAME, "markets-donut-chart__title")
                    )
                ).text
                stats["Number of analysts"] = number_of_analysts
            except Exception as e:
                stats["Number of analysts"] = float('nan')
                print(f"Brak danych o liczbie analityków dla spółki {symbol}.")
                print(e)
    
            # Przetwarzanie rekomendacji
            recommendation_columns = ["Buy Recommendation", "Hold Recommendation", "Sell Recommendation"]
            for recommendation in recommendation_columns:
                try:
                    key = recommendation.split()[0].lower()
                    recommendation_text = wait.until(
                        EC.presence_of_element_located(
                            (By.ID, f"markets-donut-chart__legend--key-value-{key}")
                        )
                    ).text
                    stats[recommendation] = recommendation_text
                except Exception as e:
                    stats[recommendation] = float('nan')
                    print(f"Brak danych o rekomendacji {recommendation} dla spółki {symbol}.")
                    print(e)
    
            # 11. Forecast (High/Median/Low)
            try:
                driver.execute_script("window.scrollTo(0, 2500);")
                time.sleep(1)
                forecast_chart_section = wait.until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "div.markets-forecast-chart")
                    )
                )
                svg_element = forecast_chart_section.find_element(By.CSS_SELECTOR, "svg")
    
                # High Forecast
                try:
                    high_data_g = svg_element.find_element(By.CSS_SELECTOR, "g.high-data")
                    high_price_tspans = high_data_g.find_elements(
                        By.CSS_SELECTOR,
                        "text.high-price tspan"
                    )
                    if len(high_price_tspans) >= 2:
                        stats["High Forecast"] = high_price_tspans[1].text.strip()
                    else:
                        stats["High Forecast"] = float('nan')
                except:
                    stats["High Forecast"] = float('nan')
    
                # Median Forecast
                try:
                    median_data_g = svg_element.find_element(By.CSS_SELECTOR, "g.median-data")
                    median_price_tspans = median_data_g.find_elements(
                        By.CSS_SELECTOR,
                        "text.median-price tspan"
                    )
                    if len(median_price_tspans) >= 2:
                        stats["Median Forecast"] = median_price_tspans[1].text.strip()
                    else:
                        stats["Median Forecast"] = float('nan')
                except:
                    stats["Median Forecast"] = float('nan')
    
                # Low Forecast
                try:
                    low_data_g = svg_element.find_element(By.CSS_SELECTOR, "g.low-data")
                    low_price_tspans = low_data_g.find_elements(
                        By.CSS_SELECTOR,
                        "text.median-price tspan"
                    )
                    if len(low_price_tspans) >= 2:
                        stats["Low Forecast"] = low_price_tspans[1].text.strip()
                    else:
                        stats["Low Forecast"] = float('nan')
                except:
                    stats["Low Forecast"] = float('nan')
    
            except Exception as e:
                print(f"Brak danych forecast dla spółki {symbol}.")
                print(e)
                stats["High Forecast"] = float('nan')
                stats["Median Forecast"] = float('nan')
                stats["Low Forecast"] = float('nan')
    
            # Dodaj do listy all_data
            all_data.append(stats)
    
        except Exception as e:
            print(f"Wystąpił błąd podczas przetwarzania spółki {symbol}.")
            new_drops.append(symbol)  # Dodaj symbol do new_drops zamiast do drops
            print(f"Dopisano spółkę {symbol} do listy ponownego przetworzenia.")
            print(e)
            print("-" * 57)
    
    # Zamknij driver po zakończeniu pętli
    driver.quit()
    
    # Tworzymy DataFrame z listy słowników
    df = pd.DataFrame(all_data)
    
    #  # Pobieranie aktualnego czasu UTC
    # current_time = datetime.now(timezone.utc)
    # 
    # # Wyodrębnienie daty i godziny
    # hour_timestamp = current_time.strftime("%H:%M:%S UTC")
    # date_timestamp = current_time.strftime("%Y-%m-%d")

    # Dodanie kolumn do DataFrame
    df["Time of record"] = hour_timestamp
    df["Date of record"] = date_timestamp
    
    return df, new_drops  # Zwracamy zarówno DataFrame, jak i nową listę drops


def clean_data(df):
    """
    Funkcja czyszcząca dane w DataFrame.
    
    Parametry:
    - df (pd.DataFrame): DataFrame zawierający dane do wyczyszczenia.
    
    Zwraca:
    - pd.DataFrame: Oczyszczony DataFrame.
    """
    
    # 1. Obróbka kolumny "Percent Change"
    if "Percent Change" in df.columns:
        df["Percent Change"] = df["Percent Change"].str.replace("%", "", regex=False)
        df["Percent Change"] = pd.to_numeric(df["Percent Change"], errors='coerce')  # Konwertujemy na float z obsługą błędów
        # Dodanie znaku + lub - w zależności od kolumny "Change"
        if "Change" in df.columns:
            df["Percent Change"] = df["Percent Change"] * df["Change"].str.contains(r"\+", regex=True).map({True: 1, False: -1})
    
    # 2. Ucięcie znaku "$" i zamiana na float w kolumnach "High Forecast", "Median Forecast", "Low Forecast"
    forecast_columns = ["High Forecast", "Median Forecast", "Low Forecast"]
    for col in forecast_columns:
        if col in df.columns:
            df[col] = (
                df[col]
                .str.replace('$', '', regex=False)  # Usuń znak dolara
                .str.replace(',', '', regex=False)  # Usuń przecinki
            )
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Konwertuj na float z obsługą błędów
    
    # 3. Ucięcie znaku "%" i zamiana na float w kolumnach "Buy Recommendation", "Hold Recommendation", "Sell Recommendation"
    recommendation_columns = ["Buy Recommendation", "Hold Recommendation", "Sell Recommendation"]
    for col in recommendation_columns:
        if col in df.columns:
            df[col] = df[col].str.replace('%', '', regex=False)
            df[col] = pd.to_numeric(df[col], errors='coerce') / 100.0  # Konwertuj na float i przekształć na wartości procentowe
    
    # 4. Wycięcie tekstu " analyst ratings" z kolumny "Number of analysts" i zamiana na float
    if "Number of analysts" in df.columns:
        df["Number of analysts"] = df["Number of analysts"].str.extract(r'(\d+)')
        df["Number of analysts"] = pd.to_numeric(df["Number of analysts"], errors='coerce')  # Konwertuj na float z obsługą błędów
    
    # 5. Scoring - zaawansowany algorytm
    if "Price" in df.columns:
        df["Price"] = df["Price"].str.replace(',', '', regex=False)
        df["Price"] = pd.to_numeric(df["Price"], errors='coerce')  # Konwertuj na float z obsługą błędów
    
    # 6. Usuń przecinki w kolumnie "Employees" i zamień na float
    if "Employees" in df.columns:
        df["Employees"] = df["Employees"].str.replace(",", "", regex=False)
        df["Employees"] = pd.to_numeric(df["Employees"], errors='coerce')  # Konwertuj na float z obsługą błędów
    
    # 7. Konwersja "Smart Score"
    if "Smart Score" in df.columns:
        df["Smart Score"] = pd.to_numeric(df["Smart Score"], errors='coerce')  # Konwertuj na float z obsługą błędów
    
    # 8. Calculate percentage growth for each forecast
    forecast_percent_columns = ["High Forecast Percent", "Median Forecast Percent", "Low Forecast Percent"]
    for forecast, percent_col in zip(forecast_columns, forecast_percent_columns):
        if forecast in df.columns and "Price" in df.columns:
            df[percent_col] = ((df[forecast] - df["Price"]) / df["Price"]) * 100
            df[percent_col] = pd.to_numeric(df[percent_col], errors='coerce')  # Konwertuj na float z obsługą błędów
    
    # 9. Ucięcie "x" z P/E ratio i konwersja na float
    if "P/E ratio" in df.columns:
        df["P/E ratio"] = df["P/E ratio"].astype(str).str.replace("x", "", regex=False)
        df["P/E ratio"] = pd.to_numeric(df["P/E ratio"], errors='coerce')  # Konwertuj na float z obsługą błędów
    
    # 10. Scoring - kontynuacja (po konwersji "Price")
    if "Price" in df.columns and "Score" not in df.columns:
        # Upewnij się, że wszystkie wymagane kolumny są dostępne
        required_cols = ["Low Forecast", "Median Forecast", "High Forecast", 
                         "Buy Recommendation", "Sell Recommendation", "Number of analysts"]
        if all(col in df.columns for col in required_cols):
            df["Score"] = (
                3 * ((df["Low Forecast"] / df["Price"]) - 1)
                + 2 * ((df["Median Forecast"] / df["Price"]) - 1)
                + 1 * ((df["High Forecast"] / df["Price"]) - 1)
                + (df["Buy Recommendation"] - df["Sell Recommendation"])
                + 0.02 * df["Number of analysts"]
            )
        else:
            df["Score"] = float('nan')  # Lub inna wartość domyślna
            
            
    # 11. Konwersja "Market cap" na "Market cap clear"
    if "Market cap" in df.columns:
        # Wyodrębnienie liczby i jednostki
        market_cap_split = df["Market cap"].str.extract(r'([\d\.]+)([TBM])')
        # Definicja mapowania jednostek
        multiplier = {'M': 1e6, 'B': 1e9, 'T': 1e12}
        # Przekształcenie na float z odpowiednim mnożnikiem
        df["Market cap clear"] = pd.to_numeric(market_cap_split[0], errors='coerce') * market_cap_split[1].map(multiplier)
        
    # 12. Konwersja "Dividend yield"
    if "Dividend yield" in df.columns:
        df["Dividend yield"] = df["Dividend yield"].str.replace('%', '', regex=False)
        df["Dividend yield"] = pd.to_numeric(df["Dividend yield"], errors='coerce')  # Konwertuj na float z obsługą błędów
        
    # 12. Konwersja dat
    date_columns = ["Ex-dividend date", "Dividend pay date"]
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce').dt.strftime('%Y-%m-%d')
    
    # --- Porządkowanie Kolejności Kolumn ---
    ordered_columns = [
        'Stock', 'Price', 'Change', 'Percent Change', 'Closing Price', 'Sector', 
        'Industry', 'Employees', 'Founded', 'Website', 
        '1-day range', '52-week range', 'Market cap', "Market cap clear", 'P/E ratio',
        'Next earnings date', 'Dividend yield', 'Ex-dividend date', 'Dividend pay date',
        'Number of analysts', 'Buy Recommendation', 'Hold Recommendation', 'Sell Recommendation', 
        'Smart Score', 'Score', 'High Forecast', "High Forecast Percent", 
        'Median Forecast', "Median Forecast Percent", 
        'Low Forecast', "Low Forecast Percent",
        'Total revenue Value', 'Total revenue Change', 
        'Net income Value', 'Net income Change',
        'Earnings per share Value', 'Earnings per share Change',
        'Net profit margin Value', 'Net profit margin Change', 
        'Free cash flow Value', 'Free cash flow Change', 
        'Debt-to-equity ratio Value', 'Debt-to-equity ratio Change', 
        'Date of record', 'Time of record', 
        "Page Load Time (s)", "Dynamic Element Load Time (s)", "Fear & Greed Index" 
    ]
    
    # Dostosowanie kolejności kolumn, jeśli istnieją
    existing_columns = [col for col in ordered_columns if col in df.columns]
    df = df[existing_columns]
    
    return df


def scrape_fear_greed_index(driver_path):
    
    # Inicjalizacja danych
    fgi_data = {
        "Fear & Greed Index": None,
        "FGI Time": None
    }

    try:
        # Konfiguracja Selenium
        service = Service(driver_path)
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Opcjonalnie: uruchomienie w trybie headless
        driver = webdriver.Chrome(service=service, options=options)
        wait = WebDriverWait(driver, 30)

        # Nawigacja do strony Fear & Greed Index
        url = "https://edition.cnn.com/markets/fear-and-greed"
        print(f"Naviguję do strony: {url}")
        driver.get(url)

        # Czekanie na załadowanie elementu z wartością Fear & Greed Index
        print("Czekam na załadowanie elementu z wartością Fear & Greed Index...")
        element = wait.until(
            EC.presence_of_element_located(
                (By.CLASS_NAME, "market-fng-gauge__dial-number-value")
            )
        )

        # Pobranie wartości
        fgi_value = element.text.strip()
        print(f"Znaleziono Fear & Greed Index: {fgi_value}")

        # Konwersja do liczby całkowitej
        try:
            fgi_value = int(fgi_value)
            fgi_data["Fear & Greed Index"] = fgi_value
        except ValueError:
            print(f"Nie udało się przekonwertować wartości FGI na int: '{fgi_value}'")

        # Pobranie aktualnego czasu UTC
        fgi_data["FGI Time"] = current_time.strftime("%Y-%m-%d %H:%M:%S UTC")
        print(f"FGI Time: {fgi_data['FGI Time']}")

    except Exception as e:
        print("Wystąpił błąd podczas pobierania Fear & Greed Index:")
        print(e)
    finally:
        # Zamknięcie przeglądarki
        driver.quit()

    return fgi_data


# Uruchamianie funkcji
df, drops = scrape_data(limit=502)

print("Spółki do ponownego przetworzenia:")
for drop in drops:
    print(drop)

if drops:
    retry_data, new_drops = retry_drops(drops)
    df_retry = pd.DataFrame(retry_data)
    df = pd.concat([df, df_retry], ignore_index=True)

    if new_drops:
        print("Spółki, które nadal nie zostały przetworzone po retry:")
        for drop in new_drops:
            print(drop)
    else:
        print("Wszystkie spółki zostały przetworzone po retry.")
else:
    print("Brak spółek do ponownego przetworzenia.")
    print("")
    print("-" * 57)
    
    
#Uruchomienie funkcji pobierającej Fear & Greed Index
fgi_data = scrape_fear_greed_index(driver_path)

#Dodanie danych FGI do DataFrame
if fgi_data["Fear & Greed Index"] is not None:
    df["Fear & Greed Index"] = fgi_data["Fear & Greed Index"]
    # df["FGI Time"] = fgi_data["FGI Time"]
else:
    df["Fear & Greed Index"] = float('nan')
    # df["FGI Time"] = None

df = clean_data(df)

# Zapisz DataFrame do pliku CSV
# Sprawdź, czy plik już istnieje
file_exists = os.path.isfile(OUTPUT_CSV_PATH)
df.to_csv(
    OUTPUT_CSV_PATH,
    index=False,
    mode='a',  # 'a' dla append (dopisywanie), 'w' dla write (pisanie od nowa)
    header=not file_exists,  # Zapisz nagłówek tylko jeśli plik nie istnieje
    sep=';'
)

print(df)

# def job():
#     """
#     Funkcja, którą schedule będzie uruchamiać raz na dobę o 23:30 UTC.
#     Pobiera DataFrame i zapisuje do pliku CSV.
#     """
#     df = scrape_data()
# 
#     # Zapis do pliku CSV
#     # Sprawdzenie, czy plik istnieje, aby dodać nagłówki tylko raz
#     try:
#         with open(OUTPUT_CSV_PATH, 'x', encoding='utf-8') as f:
#             df.to_csv(f, index=False)
#     except FileExistsError:
#         df.to_csv(OUTPUT_CSV_PATH, index=False, mode='a', header=False)
# 
#     print("Zapisano dane do CSV:", OUTPUT_CSV_PATH)
#     print(df)
# 
# # --- [HARMONOGRAM] ---
# # Ustawiamy, że job() ma się uruchamiać codziennie o 23:30 (UTC)
# schedule.every().day.at("23:30").do(job)
# 
# # Pętla nasłuchująca - tu skrypt będzie działał non-stop
# if __name__ == "__main__":
#     print("Rozpoczynam działanie harmonogramu. Czekam na zadanie...")
#     while True:
#         schedule.run_pending()
#         time.sleep(60)  # sprawdzaj zadania co minutę


Wszystkie elementy w liście są unikalne.
Liczba monitorowanych spółek - po obróbce duplikatów: 502

---------------------------------------------------------
Procesuję spółkę 1/502: AAPL
Czas ładowania strony AAPL: 3.76 sekund
Czas oczekiwania na dynamiczne elementy AAPL: 0.01 sekund

---------------------------------------------------------
Procesuję spółkę 2/502: NVDA
Czas ładowania strony NVDA: 1.75 sekund
Czas oczekiwania na dynamiczne elementy NVDA: 0.01 sekund

---------------------------------------------------------
Procesuję spółkę 3/502: MSFT
Czas ładowania strony MSFT: 2.70 sekund
Czas oczekiwania na dynamiczne elementy MSFT: 0.01 sekund

---------------------------------------------------------
Procesuję spółkę 4/502: AMZN
Czas ładowania strony AMZN: 1.63 sekund
Czas oczekiwania na dynamiczne elementy AMZN: 0.01 sekund

---------------------------------------------------------
Procesuję spółkę 5/502: GOOGL
Czas ładowania strony GOOGL: 2.14 sekund
Czas oczekiwania na dynamic

In [9]:
fgi_data = scrape_fear_greed_index(driver_path)

Naviguję do strony: https://edition.cnn.com/markets/fear-and-greed
Czekam na załadowanie elementu z wartością Fear & Greed Index...
Znaleziono Fear & Greed Index: 27
FGI Time: 2025-01-14 08:51:44 UTC


In [10]:
df

,Stock,Price,Change,Percent Change,Closing Price,Sector,Industry,Employees,Founded,Website,...,Net profit margin Change,Free cash flow Value,Free cash flow Change,Debt-to-equity ratio Value,Debt-to-equity ratio Change,Date of record,Time of record,Page Load Time (s),Dynamic Element Load Time (s),Fear & Greed Index
0,AAPL,234.40,- 2.45,-1.03,235.45,Electronic Technology,Telecommunications Equipment,164000,1976,www.apple.com,...,39.48%,23.90B,15.51%,2.09,4.83%,2025-01-14,08:51:44 UTC,3.76,0.01,27
1,NVDA,133.23,- 2.68,-1.97,134.93,Electronic Technology,Semiconductors,29600,1993,www.nvidia.com,...,7.90%,16.81B,138.36%,0.16,53.19%,2025-01-14,08:51:44 UTC,1.75,0.01,27
2,MSFT,417.19,- 1.76,-0.42,418.55,Technology Services,Packaged Software,228000,1975,www.microsoft.com,...,4.64%,19.26B,8.15%,0.34,29.71%,2025-01-14,08:51:44 UTC,2.70,0.01,27
3,AMZN,218.46,- 0.48,-0.22,219.40,Retail Trade,Internet Retail,1525000,1994,www.aboutamazon.com,...,39.73%,3.35B,61.65%,0.58,30.70%,2025-01-14,08:51:44 UTC,1.63,0.01,27
4,GOOGL,191.01,- 1.03,-0.54,192.35,Technology Services,Internet Software/Services,182502,2015,www.abc.xyz,...,16.09%,17.64B,21.96%,0.09,13.49%,2025-01-14,08:51:44 UTC,2.14,0.01,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,FSLR,183.61,- 5.63,-2.98,185.50,Electronic Technology,Semiconductors,6700,1999,www.firstsolar.com,...,5.23%,- 487.65M,303.75%,0.09,6.03%,2025-01-14,08:51:44 UTC,1.95,0.01,27
497,KIM,22.07,+ 0.41,1.89,22.55,Finance,Real Estate Investment Trusts,660,1973,www.kimcorealty.com,...,0.98%,211.08M,24.92%,0.80,6.35%,2025-01-14,08:51:44 UTC,2.69,0.01,27
498,TXT,76.97,+ 1.32,1.74,78.00,Electronic Technology,Aerospace & Defense,35000,1923,www.textron.com,...,19.13%,136.00M,24.02%,0.57,3.10%,2025-01-14,08:51:44 UTC,2.18,0.01,27
499,APTV,60.61,+ 1.26,2.12,62.69,Producer Manufacturing,Electrical Products,154000,n/a,www.aptiv.com,...,76.52%,326.00M,38.95%,1.14,75.01%,2025-01-14,08:51:44 UTC,2.51,0.01,27


In [24]:
df[(df["Stock"] == "BIIB")]["Low Forecast Percent"]

344    18.524437
Name: Low Forecast Percent, dtype: float64

In [19]:
#14.01.2025
df["Smart Score"] = df["Smart Score"].astype(float)
scoring = df[["Stock", "Price", "Low Forecast Percent", "High Forecast Percent", "Smart Score", "Score",
              "P/E ratio"]].sort_values("Score", ascending=False, ignore_index=True)
scoring[(scoring["Smart Score"] > 7) & (scoring["Score"] > 1.5) & (scoring["Low Forecast Percent"] > 0)]

,Stock,Price,Low Forecast Percent,High Forecast Percent,Smart Score,Score,P/E ratio
8,MGM,32.33,29.910300,73.213733,9.0,4.084411,11.56
9,UBER,65.70,14.155251,82.648402,9.0,4.030868,32.16
14,SLB,39.85,22.961104,63.111669,9.0,3.690489,12.83
19,ADBE,408.50,7.711138,72.472460,8.0,3.446034,33.05
22,BLDR,146.47,19.478391,58.394210,9.0,3.342322,14.32
23,LKQ,36.49,28.802412,64.428611,9.0,3.318835,13.51
28,HAL,27.80,22.302158,54.676259,8.0,3.207698,9.70
30,MSFT,417.19,1.872049,55.804310,8.0,3.121194,34.58
31,WYNN,81.33,5.742039,78.285995,10.0,3.112288,9.83
33,AMTM,22.82,31.463628,35.845749,8.0,3.015460,NaN


In [11]:
#13.01.2025
df["Smart Score"] = df["Smart Score"].astype(float)
scoring = df[["Stock", "Price", "Low Forecast Percent", "High Forecast Percent", "Smart Score", "Score",
              "P/E ratio"]].sort_values("Score", ascending=False, ignore_index=True)
scoring[(scoring["Smart Score"] > 8) & (scoring["Score"] > 1.5) & (scoring["Low Forecast Percent"] > 0)]

,Stock,Price,Low Forecast Percent,High Forecast Percent,Smart Score,Score,P/E ratio
8,MGM,32.33,29.910300,73.213733,9.0,4.084411,11.56
9,UBER,65.70,14.155251,82.648402,9.0,4.030868,32.16
14,SLB,39.85,22.961104,63.111669,9.0,3.690489,12.83
22,BLDR,146.47,19.478391,58.394210,9.0,3.342322,14.32
23,LKQ,36.49,28.802412,64.428611,9.0,3.318835,13.51
31,WYNN,81.33,5.742039,78.285995,10.0,3.112288,9.83
34,MPWR,587.35,3.856304,87.281859,9.0,3.000987,66.19
35,AMT,176.94,21.510116,44.116650,9.0,2.993602,74.59
40,COP,104.23,10.332918,58.303751,10.0,2.923856,12.38
43,PHM,108.99,10.101844,78.915497,10.0,2.774756,8.03


In [17]:
stocks

['https://edition.cnn.com/markets/stocks/AAPL',
 'https://edition.cnn.com/markets/stocks/NVDA',
 'https://edition.cnn.com/markets/stocks/MSFT',
 'https://edition.cnn.com/markets/stocks/AMZN',
 'https://edition.cnn.com/markets/stocks/GOOGL',
 'https://edition.cnn.com/markets/stocks/GOOG',
 'https://edition.cnn.com/markets/stocks/META',
 'https://edition.cnn.com/markets/stocks/TSLA',
 'https://edition.cnn.com/markets/stocks/AVGO',
 'https://edition.cnn.com/markets/stocks/BRKR',
 'https://edition.cnn.com/markets/stocks/WMT',
 'https://edition.cnn.com/markets/stocks/LLY',
 'https://edition.cnn.com/markets/stocks/JPM',
 'https://edition.cnn.com/markets/stocks/V',
 'https://edition.cnn.com/markets/stocks/MA',
 'https://edition.cnn.com/markets/stocks/ORCL',
 'https://edition.cnn.com/markets/stocks/XOM',
 'https://edition.cnn.com/markets/stocks/UNH',
 'https://edition.cnn.com/markets/stocks/COST',
 'https://edition.cnn.com/markets/stocks/PG',
 'https://edition.cnn.com/markets/stocks/HD',
 'htt

In [12]:
#10.01.2025
df["Smart Score"] = df["Smart Score"].astype(float)
scoring = df[["Stock", "Price", "Low Forecast Percent", "High Forecast Percent", "Smart Score", "Score",
              "P/E ratio"]].sort_values("Score", ascending=False, ignore_index=True)
scoring[(scoring["Smart Score"] > 7) & (scoring["Score"] > 1.5) & (scoring["Low Forecast Percent"] > 0)]

,Stock,Price,Low Forecast Percent,High Forecast Percent,Smart Score,Score,P/E ratio
8,MGM,32.33,29.910300,73.213733,9.0,4.084411,11.56
9,UBER,65.70,14.155251,82.648402,9.0,4.030868,32.16
14,SLB,39.85,22.961104,63.111669,9.0,3.690489,12.83
19,ADBE,408.50,7.711138,72.472460,8.0,3.446034,33.05
22,BLDR,146.47,19.478391,58.394210,9.0,3.342322,14.32
23,LKQ,36.49,28.802412,64.428611,9.0,3.318835,13.51
28,HAL,27.80,22.302158,54.676259,8.0,3.207698,9.70
30,MSFT,417.19,1.872049,55.804310,8.0,3.121194,34.58
31,WYNN,81.33,5.742039,78.285995,10.0,3.112288,9.83
33,AMTM,22.82,31.463628,35.845749,8.0,3.015460,NaN


In [13]:
#09.01.2025
df["Smart Score"] = df["Smart Score"].astype(float)
scoring = df[["Stock", "Price", "Low Forecast Percent", "High Forecast Percent", "Smart Score", "Score",
              "P/E ratio"]].sort_values("Score", ascending=False, ignore_index=True)
scoring[(scoring["Smart Score"] > 8) & (scoring["Score"] > 1.5) & (scoring["Low Forecast Percent"] > 0)]

,Stock,Price,Low Forecast Percent,High Forecast Percent,Smart Score,Score,P/E ratio
8,MGM,32.33,29.910300,73.213733,9.0,4.084411,11.56
9,UBER,65.70,14.155251,82.648402,9.0,4.030868,32.16
14,SLB,39.85,22.961104,63.111669,9.0,3.690489,12.83
22,BLDR,146.47,19.478391,58.394210,9.0,3.342322,14.32
23,LKQ,36.49,28.802412,64.428611,9.0,3.318835,13.51
31,WYNN,81.33,5.742039,78.285995,10.0,3.112288,9.83
34,MPWR,587.35,3.856304,87.281859,9.0,3.000987,66.19
35,AMT,176.94,21.510116,44.116650,9.0,2.993602,74.59
40,COP,104.23,10.332918,58.303751,10.0,2.923856,12.38
43,PHM,108.99,10.101844,78.915497,10.0,2.774756,8.03


In [14]:
#08.01.2025
df["Smart Score"] = df["Smart Score"].astype(float)
scoring = df[["Stock", "Price", "Low Forecast Percent", "High Forecast Percent", "Smart Score", "Score",
              "P/E ratio"]].sort_values("Score", ascending=False, ignore_index=True)
scoring[(scoring["Smart Score"] > 7) & (scoring["Score"] > 1.5) & (scoring["Low Forecast Percent"] > 0)]

,Stock,Price,Low Forecast Percent,High Forecast Percent,Smart Score,Score,P/E ratio
8,MGM,32.33,29.910300,73.213733,9.0,4.084411,11.56
9,UBER,65.70,14.155251,82.648402,9.0,4.030868,32.16
14,SLB,39.85,22.961104,63.111669,9.0,3.690489,12.83
19,ADBE,408.50,7.711138,72.472460,8.0,3.446034,33.05
22,BLDR,146.47,19.478391,58.394210,9.0,3.342322,14.32
23,LKQ,36.49,28.802412,64.428611,9.0,3.318835,13.51
28,HAL,27.80,22.302158,54.676259,8.0,3.207698,9.70
30,MSFT,417.19,1.872049,55.804310,8.0,3.121194,34.58
31,WYNN,81.33,5.742039,78.285995,10.0,3.112288,9.83
33,AMTM,22.82,31.463628,35.845749,8.0,3.015460,NaN


In [15]:
df[df["High Forecast Percent"] > 0].sort_values("High Forecast Percent", ascending=False, ignore_index=True)

,Stock,Price,Change,Percent Change,Closing Price,Sector,Industry,Employees,Founded,Website,...,Net profit margin Change,Free cash flow Value,Free cash flow Change,Debt-to-equity ratio Value,Debt-to-equity ratio Change,Date of record,Time of record,Page Load Time (s),Dynamic Element Load Time (s),Fear & Greed Index
0,TSCO,54.57,- 0.02,-0.04,54.25,Retail Trade,Specialty Stores,50000,1938,www.tractorsupply.com,...,6.84%,- 101.93M,373.40%,2.33,1.16%,2025-01-14,08:51:44 UTC,1.85,0.01,27
1,MRNA,35.15,- 7.10,-16.80,34.81,Health Technology,Biotechnology,5600,2010,www.modernatx.com,...,100.35%,- 1.72B,1.32%,0.11,5.06%,2025-01-14,08:51:44 UTC,3.52,0.01,27
2,SMCI,31.08,- 1.52,-4.66,31.91,Electronic Technology,Computer Processing Hardware,5126,1993,www.supermicro.com,...,25.00%,n/a,n/a,0.40,170.02%,2025-01-14,08:51:44 UTC,2.13,0.01,27
3,PANW,167.82,- 5.60,-3.23,168.99,Technology Services,Packaged Software,15289,2005,www.paloaltonetworks.com,...,58.58%,1.47B,198.17%,0.17,83.06%,2025-01-14,08:51:44 UTC,1.78,0.01,27
4,MU,95.06,- 4.28,-4.31,96.30,Electronic Technology,Semiconductors,48000,1978,www.micron.com,...,182.23%,38.00M,90.40%,0.31,6.58%,2025-01-14,08:51:44 UTC,1.77,0.01,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,JBL,153.51,- 0.37,-0.24,153.56,Producer Manufacturing,Industrial Machinery,138000,1966,www.jabil.com,...,38.19%,215.00M,47.43%,2.07,61.83%,2025-01-14,08:51:44 UTC,3.31,0.01,27
493,JNPR,38.49,+ 0.04,0.10,38.49,Technology Services,Information Technology Services,11144,1996,www.juniper.net,...,27.79%,162.10M,43.79%,0.38,2.41%,2025-01-14,08:51:44 UTC,2.09,0.01,27
494,WAT,399.87,+ 19.52,5.13,404.99,Health Technology,Medical Specialties,7900,1958,www.waters.com,...,15.39%,178.96M,49.42%,1.19,58.49%,2025-01-14,08:51:44 UTC,2.10,0.03,27
495,K,81.51,+ 0.00,0.00,81.35,Consumer Non-Durables,Food: Specialty/Candy,23000,1906,www.kellanova.com,...,66.22%,410.00M,30.39%,1.78,6.57%,2025-01-14,08:51:44 UTC,2.83,0.01,27


In [16]:
df["Smart Score"] = df["Smart Score"].astype(float)
scoring = df[["Stock", "Price", "Low Forecast Percent", "High Forecast Percent", "Smart Score", "Score",
              "P/E ratio"]].sort_values("Score", ascending=False, ignore_index=True)
scoring[(scoring["Smart Score"] > 7) & (scoring["Score"] > 1.5) & (scoring["Low Forecast Percent"] > 0)]

,Stock,Price,Low Forecast Percent,High Forecast Percent,Smart Score,Score,P/E ratio
8,MGM,32.33,29.910300,73.213733,9.0,4.084411,11.56
9,UBER,65.70,14.155251,82.648402,9.0,4.030868,32.16
14,SLB,39.85,22.961104,63.111669,9.0,3.690489,12.83
19,ADBE,408.50,7.711138,72.472460,8.0,3.446034,33.05
22,BLDR,146.47,19.478391,58.394210,9.0,3.342322,14.32
23,LKQ,36.49,28.802412,64.428611,9.0,3.318835,13.51
28,HAL,27.80,22.302158,54.676259,8.0,3.207698,9.70
30,MSFT,417.19,1.872049,55.804310,8.0,3.121194,34.58
31,WYNN,81.33,5.742039,78.285995,10.0,3.112288,9.83
33,AMTM,22.82,31.463628,35.845749,8.0,3.015460,NaN
